### This notebook was run in Colab

#### Install libraries

In [2]:
!pip install torch torchvision
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 786.5 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 19.4 MB/s eta 0:00:00


In [3]:
from simpletransformers.classification import ClassificationModel
import numpy as np
import torch
from scipy.special import softmax
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

# mount the google drive 
from google.colab import drive
drive.mount('/content/gdrive')

warnings.filterwarnings('ignore')

# Stance dataset used for fine-tuning is from the paper: Towards Understanding and Answering Comparative Questions
# Available for download from the Touché data: https://zenodo.org/record/6873567#.Y_XFghzMJhE

data_df = pd.read_csv('gdrive/MyDrive/stance-dataset.tsv', sep='\t', encoding='utf-8') # specify the stance dataset in the google drive
data_df = data_df[data_df['object_count'] == 2]

Mounted at /content/gdrive


In [4]:
len(data_df)

961

### Fine-tune RoBERTa

In [52]:
# Use sentiment-promted RoBERTa with masked comparison objects

def transform_train(df):
    df["text_a"] = ['FIRST_ENTITY is good'] * len(df)
    df["text_b"] = df["masked_all"] # the part after the [SEP] token
    df["labels"] = df["answer_stance"]
    return df

def get_classification_model(model):
    label_count = 4
    return ClassificationModel(model[0], model[1], num_labels=label_count, use_cuda=torch.cuda.is_available(), cuda_device=0)


# any available model at simpletransformers can be used for experiments

full_model = get_classification_model(['roberta', 'roberta-large'])
train_df = transform_train(data_df).sample(frac=1) #take the whole dataset

args = {"overwrite_output_dir": True, "output_dir": "gdrive/MyDrive/checkpoints", "num_train_epochs": 10, "fp16": False, "train_batch_size": 4, "gradient_accumulation_steps": 4, "evaluate_during_training": False, 
        "learning_rate": 2e-5, "early_stopping_consider_epochs": True, "reprocess_input_data": True,
        'save_eval_checkpoints': False, 'save_model_every_epoch' : False,	'save_optimizer_and_scheduler': False, 	'save_steps': -1, "weight": [5, 1, 1, 1]}
full_model.train_model(train_df, args=args)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

  0%|          | 0/961 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/241 [00:00<?, ?it/s]

(600, 0.6594411918494734)

### Mask the comparison objects in Touché documents

In [53]:
import pandas as pd
import os
import re

def mask_text(object1, object2, text):
    l = re.findall(r"[\w']+|[.,!?;:]", text)
    object1 = object1.capitalize()
    object2 = object2.capitalize()
    if object1 in l:
        indices = [i for i, x in enumerate(l) if x == object1]
        for i in indices:
            l[i] = 'FIRST_ENTITY'
    if object1+'s' in l:
        indices = [i for i, x in enumerate(l) if x == object1+'s']
        for i in indices:
            l[i] = 'FIRST_ENTITY'
    if object1.lower() in l:
        indices = [i for i, x in enumerate(l) if x == object1.lower()]
        for i in indices:
            l[i] = 'FIRST_ENTITY'
    if object1.lower()+'s' in l:
        indices = [i for i, x in enumerate(l) if x == object1.lower()+'s']
        for i in indices:
            l[i] = 'FIRST_ENTITY'
    if object2 in l:
        indices = [i for i, x in enumerate(l) if x == object2]
        for i in indices:
            l[i] = 'SECOND_ENTITY'
    if object2+'s' in l:
        indices = [i for i, x in enumerate(l) if x == object2+'s']
        for i in indices:
            l[i] = 'SECOND_ENTITY'
    if object2.lower() in l:
        indices = [i for i, x in enumerate(l) if x == object2.lower()]
        for i in indices:
            l[i] = 'SECOND_ENTITY'
    if object2.lower()+'s' in l:
        indices = [i for i, x in enumerate(l) if x == object2.lower()+'s']
        for i in indices:
            l[i] = 'SECOND_ENTITY'
    out_text = " ".join(l)
    return re.sub(r' (?=\W)', '', out_text)

In [54]:
# Available for download from the Touché data: https://zenodo.org/record/6873567#.Y_XFghzMJhE (original dataset is touche-task2-passages-version-002.jsonl.gz)
# We converted it to csv (in this repositore in ../data/touche-to-classify.csv)

PATH_RUNS ="gdrive/MyDrive/stance"

run_df = pd.read_csv(os.path.join(PATH_RUNS, "touche-to-classify.csv"))
run_df['text_masked'] = run_df.apply(lambda x: mask_text(x['object1'], x['object2'], x['text']), axis=1)
replace_values = {'FIRST' : 2, 'SECOND' : 3, 'NO' : 0, 'NEUTRAL' : 1} # convert stance labels into numerical values
run_df = run_df.replace({"Score": replace_values}).rename({"qid": "Topic", "docno": "ID"}, axis=1)
print(len(run_df))
run_df.head(1)

2107


,Topic,ID,Score,text,Topic,topic_text,description,narrative,object1,object2,text_masked
0,12,clueweb12-0002wb-18-34442___2,0,Divided by length ostensibly to fit the amount...,12,Train or plane? Which is the better choice?,A frequent traveler (domestic and internationa...,A highly relevant document will contrast the e...,Train,plane,Divided by length ostensibly to fit the amount...


### Predict the stance

In [55]:
def transform_df(df):
    df["text_a"] = ['FIRST_ENTITY is good'] * len(df)
    df["text_b"] = df["text_masked"] # the part after the [SEP] token
    df["labels"] = df["Score"]
    return df

pred_df = transform_df(run_df)
_, raw_outputs, _ = full_model.eval_model(pred_df)
pred_probs = softmax(raw_outputs, axis=1)
pred_preds = np.argmax(raw_outputs, axis=1)

  0%|          | 0/2107 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/264 [00:00<?, ?it/s]

In [49]:
# Save the prediction results

run_df['preds'] = pred_preds
run_df.to_csv(os.path.join(PATH_RUNS, "touche-to-classify-res.csv"))